# Comparison with EventDisplay

**Purpose of this notebook:**

Compare IRF and Sensitivity as computed by pyirf and EventDisplay on the same DL2 results

**Notes:**

The following results correspond to:

- Paranal site
- Zd 20 deg, Az 180 deg
- 50 h observation time

**Resources:**

_EventDisplay_ DL2 data, https://forge.in2p3.fr/projects/cta_analysis-and-simulations/wiki/Eventdisplay_Prod3b_DL2_Lists


Download and unpack the data using 

```bash
$ curl -fL -o data.zip https://nextcloud.e5.physik.tu-dortmund.de/index.php/s/Cstsf8MWZjnz92L/download
$ unzip data.zip
$ mv eventdisplay_dl2 data
```

## Table of contents

* [Optimized cuts](#Optimized-cuts)
    - [Direction cut](#Direction-cut)
* [Differential sensitivity from cuts optimization](#Differential-sensitivity-from-cuts-optimization)
* [IRFs](#IRFs)
    - [Effective area](#Effective-area)
    - [Point Spread Function](#Point-Spread-Function)
        + [Angular resolution](#Angular-resolution)
    - [Energy dispersion](#Energy-dispersion)
        + [Energy resolution](#Energy-resolution)
    - [Background rate](#Background-rate)

## Imports

In [ ]:
import os

import numpy as np
import uproot
from astropy.io import fits
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.table import QTable

%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (9, 6)

## Input data

### _EventDisplay_

The input data provided by _EventDisplay_ is stored in _ROOT_ format, so _uproot_ is used to transform it into _numpy_ objects. 

In [ ]:
# Path of EventDisplay IRF data in the user's local setup
# Please, empty the indir_EventDisplay variable before pushing to the repo
indir = "../data/"
irf_file_event_display = "DESY.d20180113.V3.ID0_180degNIM2LST4MST4SST4SCMST4.prod3b-paranal20degs05b-NN.S.3HB9-FD.180000s.root"

irf_eventdisplay = uproot.open(os.path.join(indir, irf_file_event_display))

### Setup of output data

## _pyirf_

The following is the current IRF + sensititivy output FITS format provided by this software.

Run `python examples/calculate_eventdisplay_irfs.py` after downloading the data

In [ ]:
pyirf_file = '../pyirf_eventdisplay.fits.gz'

## Optimized cuts
[back to top](#Table-of-contents)

### Direction cut
[back to top](#Table-of-contents)

In [ ]:
from astropy.table import Table


theta_cut = Table.read(pyirf_file, hdu='THETA_CUTS_OPT')[1:-1]


theta_cut_ed = irf_eventdisplay['ThetaCut;1']
plt.errorbar(
    10**theta_cut_ed.edges[:-1],
    theta_cut_ed.values**2,
    xerr=np.diff(10**theta_cut_ed.edges),
    ls='',
    label='EventDisplay',
)

plt.errorbar(
    0.5 * (theta_cut['low'] + theta_cut['high']),
    theta_cut['cut'].quantity.to_value(u.deg)**2,
    xerr=0.5 * (theta_cut['high'] - theta_cut['low']),
    ls='',
    label='pyirf',
)

plt.legend()
plt.ylabel('θ²-cut / deg²')
plt.xlabel(r'$E_\mathrm{reco} / \mathrm{TeV}$')
plt.xscale('log')
plt.yscale('log')

## Differential sensitivity from cuts optimization
[back to top](#Table-of-contents)

In [ ]:
# [1:-1] removes under/overflow bin
sensitivity = QTable.read(pyirf_file, hdu='SENSITIVITY')[1:-1]
sensitivity

In [ ]:
plt.figure(figsize=(12,8))

# Get data from event display file
h = irf_eventdisplay["DiffSens"]
bins = 10**h.edges
x = 0.5 * (bins[:-1] + bins[1:])
width = np.diff(bins)
y = h.values

plt.errorbar(
    x,
    y, 
    xerr=width/2,
    yerr=None,
    label="EventDisplay",
    ls=''
)

unit = u.Unit('erg cm-2 s-1')


e = sensitivity['reco_energy_center']
s = (e**2 * sensitivity['flux_sensitivity'])

plt.errorbar(
    e.to_value(u.TeV),
    s.to_value(unit),
    xerr=(sensitivity['reco_energy_high'] - sensitivity['reco_energy_low']).to_value(u.TeV) / 2,
    ls='',
    label='pyirf'
)



# Style settings
plt.title('Minimal Flux Needed for 5σ Detection in 50 hours')
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel(rf"$(E^2 \cdot \mathrm{{Flux Sensitivity}}) /$ ({unit.to_string('latex')})")
plt.grid(which="both")
plt.legend()

## IRFs
[back to top](#Table-of-contents)

### Effective area
[back to top](#Table-of-contents)

In [ ]:

# Data from EventDisplay
h = irf_eventdisplay["EffectiveAreaEtrue"]

x = 0.5 * (10**h.edges[:-1] + 10**h.edges[1:])
xerr = 0.5 * np.diff(10**h.edges)
y = h.values
yerr = np.sqrt(h.variances)

plt.errorbar(x, y, xerr=xerr, yerr=yerr, ls='', label="EventDisplay")

for name in ('', '_NO_CUTS', '_ONLY_GH', '_ONLY_THETA'):

    area = QTable.read(pyirf_file, hdu='EFFECTIVE_AREA' + name)[1:-1]

    
    plt.errorbar(
        0.5 * (area['true_energy_low'] + area['true_energy_high']).to_value(u.TeV),
        area['effective_area'].to_value(u.m**2),
        xerr=0.5 * (area['true_energy_high'] - area['true_energy_low']).to_value(u.TeV),
        ls='',
        label='pyirf ' + name,
    )

# Style settings
plt.xscale("log")
plt.yscale("log")
plt.xlabel("True energy / TeV")
plt.ylabel("Effective collection area / m²")
plt.grid(which="both")
plt.show()

### Point Spread Function
[back to top](#Table-of-contents)

In [ ]:
psf_table = QTable.read(pyirf_file, hdu='PSF')[0]
# select the only fov offset bin
psf = psf_table['psf'][:, 0, :].to_value(1 / u.sr)

offset_bins = np.append(psf_table['source_offset_low'], psf_table['source_offset_high'][-1])
phi_bins = np.linspace(0, 2 * np.pi, 1000)



# Let's make a nice 2d representation of the radially symmetric PSF
r, phi = np.meshgrid(offset_bins.to_value(u.deg), phi_bins)
x = r * np.cos(phi)
y = r * np.sin(phi)


# look at a single energy bin
# repeat values for each phi bin
image = np.tile(psf[10], (len(phi_bins) - 1, 1))
plt.pcolormesh(x, y, image)
plt.xlim(-0.25, 0.25)
plt.ylim(-0.25, 0.25)
plt.xlabel('Distance from source x')
plt.ylabel('Distance from source y')
plt.gca().set_aspect(1)

In [ ]:
# Profile
center = 0.5 * (offset_bins[1:] + offset_bins[:-1])
xerr = 0.5 * (offset_bins[1:] - offset_bins[:-1])

for bin_id in [5, 10, 30]:
    plt.errorbar(
        center.to_value(u.deg),
        psf[bin_id],
        xerr=xerr.to_value(u.deg),
        ls='',
        label=f'Energy Bin {bin_id}'
    )
    
#plt.yscale('log')
plt.legend()
plt.xlim(0, 0.25)
plt.ylabel('PSF PDF / sr⁻¹')
plt.xlabel('Distance from True Source / deg')

#### Angular resolution
[back to top](#Table-of-contents)

In [ ]:
# Data from EventDisplay
h = irf_eventdisplay["AngRes"]
x = 0.5 * (10**h.edges[:-1] + 10**h.edges[1:])
xerr = 0.5 * np.diff(10**h.edges)
y = h.values
yerr = np.sqrt(h.variances)
plt.errorbar(x, y, xerr=xerr, yerr=yerr, ls='', label="EventDisplay")

# pyirf

ang_res = QTable.read(pyirf_file, hdu='ANGULAR_RESOLUTION')[1:-1]

plt.errorbar(
    0.5 * (ang_res['true_energy_low'] + ang_res['true_energy_high']).to_value(u.TeV),
    ang_res['angular_resolution'].to_value(u.deg),
    xerr=0.5 * (ang_res['true_energy_high'] - ang_res['true_energy_low']).to_value(u.TeV),
    ls='',
    label='pyirf'
)


# Style settings
plt.xlim(1.e-2, 2.e2)
plt.ylim(2.e-2, 1)
plt.xscale("log")
plt.xlabel("True energy / TeV")
plt.ylabel("Angular Resolution / deg")
plt.grid(which="both")


plt.legend(loc="best")

### Energy dispersion
[back to top](#Table-of-contents)

In [ ]:
edisp = QTable.read(pyirf_file, hdu='EDISP')[0]
edisp

In [ ]:
e_bins = edisp['true_energy_low'][1:]
migra_bins = edisp['migration_low'][1:]

plt.title('pyirf')
plt.pcolormesh(e_bins.to_value(u.TeV), migra_bins, edisp['energy_dispersion'][1:-1, 1:-1, 0].T, cmap='inferno')

plt.xscale('log')
plt.yscale('log')
plt.colorbar(label='PDF Value')

plt.xlabel(r'$E_\mathrm{True} / \mathrm{TeV}$')
plt.ylabel(r'$E_\mathrm{Reco} / E_\mathrm{True}$')

plt.show()

#### Energy resolution
[back to top](#Table-of-contents)

In [ ]:
# Data from EventDisplay
h = irf_eventdisplay["ERes"]
x = 0.5 * (10**h.edges[:-1] + 10**h.edges[1:])
xerr = np.diff(10**h.edges) / 2
y = h.values
yerr = np.sqrt(h.variances)

# Data from pyirf
bias_resolution = QTable.read(pyirf_file, hdu='ENERGY_BIAS_RESOLUTION')[1:-1]

# Plot function
plt.errorbar(x, y, xerr=xerr, yerr=yerr, ls='', label="EventDisplay")
plt.errorbar(
    0.5 * (bias_resolution['true_energy_low'] + bias_resolution['true_energy_high']).to_value(u.TeV),
    bias_resolution['resolution'],
    xerr=0.5 * (bias_resolution['true_energy_high'] - bias_resolution['true_energy_low']).to_value(u.TeV),
    ls='',
    label='pyirf'
)
plt.xscale('log')

# Style settings
plt.xlabel(r"$E_\mathrm{True} / \mathrm{TeV}$")
plt.ylabel("Energy resolution")
plt.grid(which="both")


plt.legend(loc="best")
plt.show()

### Background rate
[back to top](#Table-of-contents)

In [ ]:
# Data from EventDisplay
h = irf_eventdisplay["BGRate"]
x = 0.5 * (10**h.edges[:-1] + 10**h.edges[1:])
xerr = np.diff(10**h.edges) / 2
y = h.values
yerr = np.sqrt(h.variances)

# Style settings
plt.xscale("log")
plt.xlabel(r"$E_\mathrm{Reco} / \mathrm{TeV}$")
plt.ylabel("Background rate / s⁻¹")
plt.grid(which="both")

# Plot function
plt.errorbar(x, y, xerr=xerr, yerr=yerr, fmt="o", label="EventDisplay")
plt.legend(loc="best")
plt.show()